In [1]:
import pysal as ps
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from numpy import linalg as la
%matplotlib inline

In [2]:
data = ps.open(ps.examples.get_path('columbus.dbf'))
W_rook = ps.open(ps.examples.get_path('columbus.gal'))
W_kernel = ps.kernelW_from_shapefile(
    ps.examples.get_path('columbus.shp'),
    3, function='gaussian', diagonal=True)
W_kernel_floatdiag = ps.kernelW_from_shapefile(
    ps.examples.get_path('columbus.shp'),
    3, function='gaussian', diagonal=False)

In [3]:
W_kernel_mat = W_kernel.full()[0]
W_kernel_mat_floatdiag = W_kernel_floatdiag.full()[0]

In [4]:
y = data.by_col_array(['HOVAL'])
X1, X2 = data.by_col_array(['INC']), data.by_col_array(['CRIME'])

In [5]:
X = np.hstack((X1, X2))

In [6]:
colshp = ps.open(ps.examples.get_path('columbus.shp')).read()
colshp_coords = np.array([x.centroid for x in colshp])

In [7]:
design = np.hstack((y, X))

In [8]:
data2 = pd.DataFrame(design, columns=['y', 'X1', 'X2'])

In [9]:
psbw = W_kernel.bandwidth[0][0]

In [10]:
psbw

0.8518467161151434

In [11]:
X2.shape, X1.shape, y.shape

((49, 1), (49, 1), (49, 1))

## Python Estimation

In [12]:
X = np.hstack((np.ones(X1.shape), X))

In [113]:
class BaseGWR(object):
    def __init__(self, y, X, W):
        self.y = y
        self.n = y.shape[0]
        self.p = X.shape[-1]
        self.X = X
        self.W = W
        self.vm = NotImplementedError
        self.betas = []
        self.yhats = []
        
        for i in range(self.n):
            Wuv = np.diag(np.sqrt(self.W.full()[0][i,:]))
            xtw = np.dot(self.X.T, Wuv)
            xtwx = np.dot(xtw, self.X)
            xtwx_inv = la.inv(xtwx)
            xtwx_inv_xtw = np.dot(xtwx_inv, xtw)
            beta_hats = np.dot(xtwx_inv_xtw, self.y)
            self.betas.append(beta_hats.T)
            yhat = np.dot(self.X[i,:], beta_hats)
            self.yhats.append(yhat[0])
        self.betas = np.array(self.betas).flatten()
        self.yhats = np.array(self.yhats).reshape(self.y.shape)
            
        self.SSE = np.sum((self.y - self.yhats)**2)
        self.MSE = self.SSE / float(self.n)
        self.SST = np.sum((self.y - np.mean(self.y))**2)
        self.SSR = self.SST - self.SSE
        self.R2 = self.SSR/self.SST
        
        self.MSR = self.SSR / self.p
        self.MST = self.SST / (self.n - self.p - 1)
        self.F = self.MSR/self.MST
        
    def __repr__(self):
        return "This is a GWR model"

In [114]:
mod = BaseGWR(y, X, W_kernel)

In [142]:
mod

This is a GWR model

In [117]:
import scipy.stats as st

In [134]:
1 - st.f.cdf(mod.F, mod.n-1, mod.n-mod.p-1)

5.3623772089395061e-14

In [139]:
classical = ps.spreg.OLS(y, np.hstack((X1,X2)))

In [140]:
classical.f_stat

(12.358198885356694, 5.063690331394925e-05)

In [141]:
mod.F

11.538967334413718